In [9]:
import pandas as pd
import numpy as np
import json

with open('C:/Users/20234635/OneDrive - TU Eindhoven/Desktop/Code/PCB_Design/RecordFile/PDA_record.json', 'r') as file:
    data = json.load(file)
RECORD_DESIGN = data
SLICE = RECORD_DESIGN

In [23]:
for i in range(len(SLICE['Record']['Track']['Net'])):
    for key, value in SLICE['Record'].items():
        if key == 'Module':
            for key, value in value.items():
                module_ref = key
                module_x = value['Position X'][i]
                module_y = value['Position Y'][i]
                module_angle = value['Angle'][i]
                if i > 0:
                    module_o = np.array([value['Position X'][i-1], value['Position Y'][i-1], value['Angle'][i-1]])
                    module_p = np.array([value['Position X'][i], value['Position Y'][i], value['Angle'][i]])
        elif key == 'Track':
            track_net = value['Net'][i]
            track_start_x = value['Start X'][i]
            track_start_y = value['Start Y'][i]
            track_end_x = value['End X'][i]
            track_end_y = value['End Y'][i]
            track_width = value['Width'][i]
            track_layer = value['Layer'][i]
            track_o = np.vstack([value['Start X'][i-1], value['Start Y'][i-1], value['End X'][i-1], value['End Y'][i-1], value['Layer'][i-1], value['Width'][i-1], value['Net'][i-1]])
            track_p = np.vstack([value['Start X'][i], value['Start Y'][i], value['End X'][i], value['End Y'][i], value['Layer'][i], value['Width'][i], value['Net'][i]])
            if track_net == ['None']:
                    break
            else:
                for j in range(len(track_net)):
                    column_is_different = True  # 假设track_p[:,j]与track_o的所有列都不相等
                    for k in range(track_o.shape[1]):  # 遍历track_o的每一列
                        if np.array_equal(track_p[:,j], track_o[:,k]):  # 如果找到相等的列
                            column_is_different = False  # 更新假设
                        break
                    if column_is_different:  # 如果track_p[:,j]与track_o的所有列都不相等
                        print(track_p[1,j])
                        print('yes')

# print(track_p) 
# print(track_p[:,0])

69.47
yes
63.12
yes
59.41
yes
42.26
yes


In [11]:
trake_net_o = SLICE['Record']['Track']['Net'][-1]
trake_start_x_o = SLICE['Record']['Track']['Start X'][-1]
trake_start_y_o = SLICE['Record']['Track']['Start Y'][-1]
trake_end_x_o = SLICE['Record']['Track']['End X'][-1]
trake_end_y_o = SLICE['Record']['Track']['End Y'][-1]
trake_width_o = SLICE['Record']['Track']['Width'][-1]
trake_layer_o = SLICE['Record']['Track']['Layer'][-1]
via_net_o = SLICE['Record']['Via']['Net'][-1]
via_pos_x_o = SLICE['Record']['Via']['Position X'][-1]
via_pos_y_o = SLICE['Record']['Via']['Position Y'][-1]
via_diameter_o = SLICE['Record']['Via']['Diameter'][-1]
trake_start_o = np.vstack([trake_start_x_o, trake_start_y_o, trake_layer_o, trake_width_o, trake_net_o])
trake_end_o = np.vstack([trake_end_x_o, trake_end_y_o, trake_layer_o, trake_width_o, trake_net_o])
via_o = np.vstack([via_pos_x_o, via_pos_y_o, via_diameter_o])

def find_via(trake_o, via_o):
    for i in range(len(trake_o[0])):
        for j in range(len(via_o[0])):
            if via_o[0,j] == None:
                break
            if (trake_o[:2,i].astype(float) == via_o[:2,j]).all():
                trake_o[2,i] = 'Via'
                trake_o[3,i] = via_o[2,j]
    return trake_o

def find_connection(wire, number_track, trake_start_o, trake_end_o, lebal):
    max_lebal = lebal
    for i in range(number_track):
        if i in lebal:
            continue
        else:
            if (wire[:3,-1] == trake_start_o[:3,i]).all():
                wire = np.hstack((wire, trake_end_o[:,i].reshape(-1,1)))
                wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal + [i])
                if len(lebal) > len(max_lebal):
                    max_lebal = lebal
            elif (wire[:3,0] == trake_end_o[:3,i]).all():
                wire = np.hstack((trake_start_o[:,i].reshape(-1,1), wire))
                wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal + [i])
                if len(lebal) > len(max_lebal):
                    max_lebal = lebal
            elif (wire[:3,-1] == trake_end_o[:3,i]).all():
                wire = np.hstack((wire, trake_start_o[:,i].reshape(-1,1)))
                wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal + [i])
                if len(lebal) > len(max_lebal):
                    max_lebal = lebal
            elif (wire[:3,0] == trake_start_o[:3,i]).all():
                wire = np.hstack((trake_end_o[:,i].reshape(-1,1), wire))
                wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal + [i])
                if len(lebal) > len(max_lebal):
                    max_lebal = lebal
            else:
                continue
    return wire, max_lebal

trake_start_o = find_via(trake_start_o, via_o)
trake_end_o = find_via(trake_end_o, via_o)

lebal = []
wire_net = []
number = 0
for i in range(len(SLICE['Record']['Track']['Net'])):
    trake_net_o = SLICE['Record']['Track']['Net'][i]
    trake_start_x_o = SLICE['Record']['Track']['Start X'][i]
    trake_start_y_o = SLICE['Record']['Track']['Start Y'][i]
    trake_end_x_o = SLICE['Record']['Track']['End X'][i]
    trake_end_y_o = SLICE['Record']['Track']['End Y'][i]
    trake_width_o = SLICE['Record']['Track']['Width'][i]
    trake_layer_o = SLICE['Record']['Track']['Layer'][i]
    via_net_o = SLICE['Record']['Via']['Net'][i]
    via_pos_x_o = SLICE['Record']['Via']['Position X'][i]
    via_pos_y_o = SLICE['Record']['Via']['Position Y'][i]
    via_diameter_o = SLICE['Record']['Via']['Diameter'][i]
    trake_start_o = np.vstack([trake_start_x_o, trake_start_y_o, trake_layer_o, trake_width_o, trake_net_o])
    trake_end_o = np.vstack([trake_end_x_o, trake_end_y_o, trake_layer_o, trake_width_o, trake_net_o])
    via_o = np.vstack([via_pos_x_o, via_pos_y_o, via_diameter_o])

    number_track = len(trake_net_o)
    trake_start_o = find_via(trake_start_o, via_o)
    trake_end_o = find_via(trake_end_o, via_o)

    wires = []
    lebal = []
    number = 0
    for j in range(len(trake_net_o)):
        if j in lebal:
            continue
        else:
            wire = trake_start_o[:,j].reshape(-1,1)
            wire = np.hstack((wire, trake_end_o[:,j].reshape(-1,1)))
            lebal = lebal + [j]
            wire, lebal = find_connection(wire, number_track, trake_start_o, trake_end_o, lebal)
            wires.append(wire.tolist())
            number += 1
    wire_net.append(wires)
print(wire_net[3])

IndexError: list index out of range

In [ ]:
    Action = {'Module':[],'Wire':[]}


    module_action = []
    step_action = []
    last_step_action = []
    for i in range(len(SLICE['Record']['Track']['Net']) - 1):
        for key, value in SLICE['Record']['Module'].items():
            module_ref = key
            module_o = np.array([value['Position X'][i], value['Position Y'][i], value['Angle'][i]])
            module_p = np.array([value['Position X'][i+1], value['Position Y'][i+1], value['Angle'][i+1]])
            if not np.array_equal(module_o, module_p):
               step_action.append([module_ref, (module_p - module_o).tolist()])

        wire_o = wire_net[i]
        wire_p = wire_net[i+1]
        for j in range(len(wire_o)):
            if not wire_o[j] in wire_p and wire_o[j][0] != [None, None]:
                step_action.append(['Wire Delect', wire_o[j]])  
        for j in range(len(wire_p)):
            if not wire_p[j] in wire_o:
                step_action.append(['Wire Add', wire_p[j]])   

        if not np.array_equal(last_step_action,step_action):
            last_step_action = step_action
        else:
           step_action.append({'None':'None'})
        module_action.append([step_action])
        step_action = []
        last_step_action = []

    Action['Module'].append(module_action)
    module_action = []

    print(wire_net[0])
    print(wire_net[1])
    print(wire_net[2])
    print(wire_net[3])
    print(wire_net[4])
    print(Action['Module'])

[[[None, None], [None, None], ['None', 'None'], [None, None], ['None', 'None']]]
[[[None, None], [None, None], ['None', 'None'], [None, None], ['None', 'None']]]
[[[None, None], [None, None], ['None', 'None'], [None, None], ['None', 'None']]]
[[['58.06', '50.11', '45.61'], ['46.0', '46.0', '41.5'], ['F.Cu', 'F.Cu', 'F.Cu'], ['0.25', '0.25', '0.25'], ['', '', '']]]
[[['58.06', '50.11', '45.61'], ['46.0', '46.0', '41.5'], ['F.Cu', 'F.Cu', 'F.Cu'], ['0.25', '0.25', '0.25'], ['', '', '']]]
[[[[['L1', [-1.6213229999999967, 10.364793000000006, 0.0]]]], [[['C8', [-2.6227909999999994, -7.560591000000002, 0.0]], ['R3', [-2.6227909999999994, -7.560591000000002, 0.0]], ['R5', [-2.6227909999999994, -7.560591000000002, 0.0]]]], [[['Wire Add', [['58.06', '50.11', '45.61'], ['46.0', '46.0', '41.5'], ['F.Cu', 'F.Cu', 'F.Cu'], ['0.25', '0.25', '0.25'], ['', '', '']]]]], [[['U1', [-2.867348000000007, 5.548202999999994, 0.0]]]]]]
